In [ ]:
from datasetSplitter import datasetSplitterStratKFold
import pandas as pd

datasett = pd.read_csv("data\processed_data.csv")
datasett_normalized = (datasett - datasett.min()) / (datasett.max() - datasett.min())

kFold, X_test, Y_test = datasetSplitterStratKFold(dataFrame=datasett_normalized)

# for a,b,c,d in kFold:
#     print(a,c)
#     print(b,d)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.nn.init as init
from torch.utils.data import TensorDataset, DataLoader

class BasicModel(nn.Module):
    def __init__(self,inFeatures,hiddens,classes,activations:list):
        super().__init__()
        if isinstance(activations,list):
            assert len(activations) <= len(hiddens)
        linearList = []
        start = inFeatures
        for size in hiddens:
            linearList.append(nn.Linear(start,size))
            start = size

        layerList = []
        for idx in range(len(linearList)):
            layerList.append(linearList[idx])
            if isinstance(activations,list):
                if idx < len(activations):
                    layerList.append(activations[idx])
                else:
                    layerList.append(activations[-1])
            else:
                layerList.append(activations)
        layerList.append(nn.Linear(size,classes))
        layerList.append(nn.Sigmoid())
        self.block = nn.Sequential(
            *layerList
        )
    
    def forward(self, x):
        return self.block(x)
        
model = BasicModel(28,[64,32,16],1,
                   [nn.ReLU()])
print(model)

for layer in model.modules():
    if isinstance(layer, (nn.Linear, nn.Conv2d)):  # Check if the layer has weights
#         init.xavier_normal_(layer.weight,gain=0.1)  # Initialize weights using Xavier Normal
#         if layer.bias is not None:
#             init.normal_(layer.bias, mean=0, std=0.05)  
        print(layer.weight,layer.bias)

device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model = model.to(device)
        
criterion = nn.BCELoss()
optimiser = optim.Adam(model.parameters(),lr=0.01)

In [ ]:

from sklearn.metrics import recall_score, accuracy_score

epochs = 100
batchSize = 64

# firstBatch = True

for fold, (train_x, test_x, train_y, test_y) in enumerate(kFold,start=1):

    #? Need to convert the pd.DataFrames to torch.tensors
    # train_x = torch.tensor(train_x.values,dtype=torch.float32)
    # train_y = torch.tensor(train_y.values,dtype=torch.float32)
    # test_x = torch.tensor(test_x.values,dtype=torch.float32) 
    # test_y = torch.tensor(test_y.values,dtype=torch.float32)

    train_dataset = TensorDataset(torch.tensor(train_x.values,dtype=torch.float32), torch.tensor(train_y.values,dtype=torch.float32))
    test_dataset = TensorDataset(torch.tensor(test_x.values,dtype=torch.float32), torch.tensor(test_y.values,dtype=torch.float32))

    train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batchSize, shuffle=False)

    model.train()
    print(f"Fold: {fold}")
    for epoch in range(epochs):
        print(f" Epoch: {epoch+1}")
        running_loss = 0.
        correct_train = 0
        total_train = 0

        for batch, (inputs, labels) in enumerate(train_loader,start=1):

            inputs, labels = inputs.to(device), labels.float().to(device)
            optimiser.zero_grad()
            outputs = model(inputs)
           
            loss = criterion(outputs, labels)
            loss.backward()
            optimiser.step()

            

            running_loss += loss.item()
            predictions = (outputs > 0.5).float().cpu()
            labels2 = labels.cpu()
            total_train += labels.size(0)
            correct_train += (predictions == labels2).sum().item()
            accuracy = (predictions == labels2).float().mean()
            accuracyScore = accuracy_score(labels2,predictions)
            recallScore = recall_score(labels2,predictions,zero_division=0.0)

            if epoch == 1 and batch == 140:
                print("Inputs:",inputs)
                print("Labels",labels)
                print("Outputs", outputs)
                print(f"AccuracyScore: {(accuracy):.2f}")
                # firstBatch = False

            # print(predictions.cpu(), labels2)

            print(f"  Batch: {batch+1}",
                  f"Loss: {(running_loss/total_train):.4f}",
                  f"AccuracyScore: {(accuracy):.2f}",
                  f"RecallScore: {(recallScore):.2f}", 
                  end="\r")
            # break
        print()

        train_accuracy = (correct_train / total_train)*100
        avg_loss = running_loss / (len(train_loader)*batchSize)

        print(f"  Loss: {avg_loss:.4f}".ljust(12),
              f"Accuracy: {train_accuracy:.2f}%".ljust(50))
        # break
    # break
    print()
